<a href="https://colab.research.google.com/github/0xrocky/master-thesis/blob/main/paper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generic imports
#!pip install tensorflow
!pip install sage
from sage import *
import numpy as np
from google.colab import drive
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
# Neural network imports
import tensorflow as tf
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import *
import chardet

  Preparing metadata (setup.py) ... done
  Created wheel for sage: filename=sage-0.0.0-py3-none-any.whl size=1078 sha256=8492ddcb2eb0d6ea38f19be405bf1ea66f1cce83530c941aaa040af4587abc77
  Stored in directory: /root/.cache/pip/wheels/55/71/d5/b02b11f6fe921d637df782b4dbaa7c522ea10632ccb5a6d74c
Successfully built sage


In [ ]:
# Macro
SIZE_SIMULATION_TRACES = 1 # number of synthetic traces
MUL_TIME = 1490 # time duration of a single multiplication
SUB_TIME = 151 # time duration of a single sub
ADD_TIME = 151 # time duration of a single sub
TAG_ADD = 1
TAG_SUB = 2
TAG_MUL = 3
TAG_END = 0 # tag of a delimitator of a modular operation
MODULO_TIME = 15 # time duration of a modulo
def percent( time_op ):
  return round( ( time_op * 10) / 100 )

In [ ]:
# Load from the GDrive folder the mean patterns
LOCAL_DRIVE = '/content/drive'
drive.mount( LOCAL_DRIVE, force_remount=True )

sub_of_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_sub_of_pattern.npy', allow_pickle=True )
sub_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_sub_pattern.npy', allow_pickle=True )
add_of_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_add_of_pattern.npy', allow_pickle=True )
add_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_add_pattern.npy', allow_pickle=True )
mul_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_mul_pattern.npy', allow_pickle=True )
sqr_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_sqr_pattern.npy', allow_pickle=True )

print( "SUB overflow mean pattern (min, max, mean, len):", np.min( sub_of_pattern ), np.max( sub_of_pattern ), np.mean( sub_of_pattern, axis=0 ), len( sub_of_pattern ) )
print( "SUB mean pattern (min, max, mean, len)         :", np.min( sub_pattern ), np.max( sub_pattern ), np.mean( sub_pattern, axis=0 ), len( sub_pattern ) )
print( "ADD overflow mean pattern (min, max, mean, len):", np.min( add_of_pattern ), np.max( add_of_pattern ), np.mean( add_of_pattern, axis=0 ), len( add_of_pattern ) )
print( "ADD mean pattern (min, max, mean, len)         :", np.min( add_pattern ), np.max( add_pattern ), np.mean( add_pattern, axis=0 ), len( add_pattern ) )
print( "MUL mean pattern (min, max, mean, len)         :", np.min( mul_pattern ), np.max( mul_pattern ), np.mean( mul_pattern, axis=0 ), len( mul_pattern ) )
print( "SQR mean pattern  (min, max, mean, len)        :", np.min( sqr_pattern ), np.max( sqr_pattern ), np.mean( sqr_pattern, axis=0 ), len( sqr_pattern ) )

Mounted at /content/drive
SUB overflow mean pattern (min, max, mean, len): -12.739841762086513 11.50184378975827 0.8494655483788998 184
SUB mean pattern (min, max, mean, len)         : -12.730101781280176 11.350560745447648 0.8084656791741394 120
ADD overflow mean pattern (min, max, mean, len): -13.274613619988617 11.053875072279535 0.9736719238216869 186
ADD mean pattern (min, max, mean, len)         : -18.16480201504563 14.576766278672247 0.8835059338571355 151
MUL mean pattern (min, max, mean, len)         : -15.8601337890625 13.431892578125 0.7690925591180787 1490
SQR mean pattern  (min, max, mean, len)        : -15.6725556640625 13.434483398437498 0.7824533051679491 1496


In [ ]:
# I choose the pattern for SHORT and LONG operations
# For SO I will use the mean pattern of adds, and for the modular SO I will use the pattern of SO plus the pattern of overflow adds
SO_TIME = SUB_TIME
LO_TIME = MUL_TIME
SO_pattern = add_pattern
SO_pattern_modulo = np.concatenate( ( add_pattern, add_of_pattern[ : percent( SO_TIME ) ] ), axis=0 )
LO_pattern = mul_pattern
LO_pattern_modulo = np.concatenate( ( mul_pattern, add_of_pattern[: percent( LO_TIME ) ] ), axis=0 )

print( len( SO_pattern ) )
print( len( LO_pattern ) )
print( len( SO_pattern_modulo ) )
print( len( LO_pattern_modulo ) )

151
1490
166
1639


In [ ]:
# SAVEZ
# Da altro Notebook
arr = np.load('/content/drive/MyDrive/ECCML/[SEC_PAT]Shared/Paper/Traces/trace_testing.npz', allow_pickle=True, mmap_mode='r' )
traces = arr['x']
print( "ndim: ", np.ndim( arr ) )
print( "shape: ", np.shape( arr ) )
print( "size: ", np.size( arr ) )
print( "len: ", len( arr ) )
print( "len: ", len( arr['x'] ) )
print( "ndim: ", np.ndim( traces ) )
print( "shape: ", np.shape( traces ) )
print( "size: ", np.size( traces ) )
print( "len: ", len( traces ) )
print( traces[ 0 ][ 0 ] )
#print( len( traces[1] ) )
#print( len( parameters ) )

ndim:  1
shape:  (3,)
size:  3
len:  3
len:  3964924
ndim:  2
shape:  (3964924, 2)
size:  7929848
len:  3964924
1


In [ ]:
# Per mia comodità costruisco delle funzioni per ottenere un sample, un tag, i cicli di clock
def get_trace_hw( trace, i ):
  return trace[ i ][ 0 ]

def get_trace_tag( trace, i ):
  return trace[ i ][ 1 ]

In [ ]:
# adding the pattern for that operation
def hamming_weight_with_pattern( hw_op, pattern_op ):
  temp = [ x[ 0 ] for x in hw_op ]
  return np.add( pattern_op, temp )

In [ ]:
# Adding Gaussian vertical noise at the hamming weight values with pattern
mean = 0
var = 1.5

def ver_noise_pattern_hamming_weight( samples ):
  return [ sample + np.random.normal( loc=mean, scale=var ) for sample in samples ]

In [ ]:
# Generate a max duration for a specific operation with a plus horizontal noise of 10%
def hor_noise( operation_time ):
  return operation_time + round( operation_time * 10 / 100 )

MUL_TIME_hnoise = hor_noise( MUL_TIME )
SUB_TIME_hnoise = hor_noise( SUB_TIME )

In [ ]:
# Populating a trace with its mul and add/sub operations
def traces_populating( trace, op_time_hnoise, modular_operation, pattern_operation, tag ):
  temp = []
  hamming_weight_pattern_vnoise = ver_noise_pattern_hamming_weight( hamming_weight_with_pattern( modular_operation, pattern_operation ) )
  # op_duration_horizontal_noise = random.randint( op_time, op_time_hnoise )
  # For each sample of operation: trace = [ [ ( hw1, tag1 ), ( hw1, tag1 ), ... ], [ ( hw2, tag2 ), ( hw2, tag2 ), ... ], ... ]
  # for j in range( op_duration_horizontal_noise ):
  #  temp += [ ( hamming_weight_pattern_vnoise[ j ], tag ) ]
  for hw in hamming_weight_pattern_vnoise:
    temp += [ ( hw, tag ) ]
  trace += [ temp ]

In [ ]:
# Conto le ADD, SUB e MUL
for trace in range( SIZE_SIMULATION_TRACES ):
  adds = 0
  subs = 0
  muls = 0
  for couple in range( len( traces ) ):
    hw = get_trace_hw( traces, couple )
    tag = get_trace_tag( traces, couple )
    if( tag == TAG_END ):
      tag_prec = get_trace_tag( traces, couple - 1 )
      if tag_prec == TAG_ADD:
        adds += 1
      elif tag_prec == TAG_SUB:
        subs += 1
      elif tag_prec == TAG_MUL:
        muls += 1
  print( "adds", adds )
  print( "subs", subs )
  print( "muls", muls )
  print( "TOT", adds+subs+muls )

adds 310
subs 2453
muls 2159
TOT 4922


In [ ]:
# Tiro via il delimitatore TAG_END che mi serviva per segnalare la fine di un'operazione modulare
final_traces = []
cont = 0

for trace in range( SIZE_SIMULATION_TRACES ):
  final_trace = []
  temp = []
  for couple in range( len( traces ) ):
    hw = get_trace_hw( traces, couple )
    tag = get_trace_tag( traces, couple )
    if tag != TAG_END:
      cont += 1
      temp += [ ( hw, tag ) ]
    else:
      final_trace += [ temp ]
      temp = []
      #print( "tag", get_trace_tag( traces, couple-1 ), cont )
      cont = 0
  final_traces += [ final_trace ]

In [ ]:
print( len ( final_traces ) )
print( len ( final_traces[ 0 ] ) )
print( final_traces[ 0 ][ :2 ] )

1
4922
[[(1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3),

In [ ]:
# Appendin the boolean TAG True/False for each sample
synthetic_traces = []
cont_true = 0
cont_false = 0

for trace in final_traces:
  for modular_operation in trace:
    duration = len( modular_operation )
    # ADD / SUB case
    if get_trace_tag( modular_operation, 0 ) == TAG_ADD or get_trace_tag( modular_operation, 0 ) == TAG_SUB:
      # modulo?
      cont_true += duration
      if( duration > SO_TIME ):
        traces_populating( synthetic_traces, None, modular_operation, SO_pattern_modulo, True )
      else:
        traces_populating( synthetic_traces, None, modular_operation, SO_pattern, True )
    # MUL case
    elif get_trace_tag( modular_operation, 0 ) == TAG_MUL:
      #if cont_false > 427344:
      #  continue
      #else:
      cont_false += duration
      # modulo?
      if( duration > LO_TIME ):
        traces_populating( synthetic_traces, None, modular_operation, LO_pattern_modulo, False )
      else:
        traces_populating( synthetic_traces, None, modular_operation, LO_pattern, False )
  random.shuffle( synthetic_traces )
print( "Samples ADD/SUB", cont_true )
print( "Samples MUL", cont_false )

Samples ADD/SUB 421848
Samples MUL 3538154


In [ ]:
print( len( synthetic_traces ) )
print( synthetic_traces[ :4 ] )

4922
[[(93.58998793425509, True), (82.968083592461, True), (76.86203580547472, True), (74.83062542760929, True), (90.75312373957557, True), (74.26610515980815, True), (83.60216673398453, True), (89.48087852374415, True), (79.80662383926291, True), (81.15269643859, True), (91.07018542029935, True), (77.55954342897918, True), (79.7554929428429, True), (88.54994578576365, True), (86.06489415451347, True), (77.88956937009567, True), (78.00712915977023, True), (94.58522325212785, True), (84.61402830386837, True), (69.91251429480779, True), (86.0137884064775, True), (88.55327911453371, True), (74.7733952685734, True), (83.20383092008849, True), (87.46433464396905, True), (84.44733006331667, True), (79.4124817760518, True), (79.90137451753766, True), (72.66710299357113, True), (78.61265862772012, True), (87.3851459806707, True), (83.66426130596417, True), (85.49872231168618, True), (75.57831345362186, True), (81.89317960089768, True), (87.91757218636316, True), (80.1882411398938, True), (89.7

In [ ]:
final_traces = []
weights_trace = []
tags_trace = []

for trace in range( SIZE_SIMULATION_TRACES ):
  temp_trace_hw = []
  temp_trace_tag = []
  for modular_operation in synthetic_traces:
    for couple_hw_tag in modular_operation:
      #weights_traces += [ couple_hw_tag[ 0 ] ]
      #tags_traces += [ couple_hw_tag[ 1 ] ]
      temp_trace_hw += [ couple_hw_tag[ 0 ] ]
      temp_trace_tag += [ couple_hw_tag[ 1 ] ]
  weights_trace += [ temp_trace_hw ]
  tags_trace += [ temp_trace_tag ]

size_weights_simulation_trace = len( weights_trace )
size_tags_simulation_trace = len( tags_trace )
size_weights_single_trace = len( weights_trace[ 0 ] )
size_tags_single_trace = len( tags_trace[ 0 ] )

print( synthetic_traces[ 0 ][:100] )
print( weights_trace[ 0 ][:100] )
print( tags_trace[ 0][:100] )

print( "Size of the list of all simulation traces (numbers of executions):", SIZE_SIMULATION_TRACES )
print( "Size of the list of all weights traces constructed from the simulation traces:", size_weights_simulation_trace )
print( "Size of the list of all tags traces constructed from the simulation traces:", size_tags_simulation_trace )
#print( "Size of the only first simulation trace", SIZE_SINGLE_TRACE )
print( "Size of the only first weight trace", size_weights_single_trace )
print( "Size of the only first tags trace", size_tags_single_trace )

if SIZE_SIMULATION_TRACES != size_weights_simulation_trace or SIZE_SIMULATION_TRACES != size_tags_simulation_trace:
  print( "ERROR: number of total weights not equal at number of total tags in the simulation traces" )
#if size_tags_single_trace != size_weights_single_trace:  print( "ERROR: number of weights of the first trace not equal at number of tags of the first trace" )

[(93.58998793425509, True), (82.968083592461, True), (76.86203580547472, True), (74.83062542760929, True), (90.75312373957557, True), (74.26610515980815, True), (83.60216673398453, True), (89.48087852374415, True), (79.80662383926291, True), (81.15269643859, True), (91.07018542029935, True), (77.55954342897918, True), (79.7554929428429, True), (88.54994578576365, True), (86.06489415451347, True), (77.88956937009567, True), (78.00712915977023, True), (94.58522325212785, True), (84.61402830386837, True), (69.91251429480779, True), (86.0137884064775, True), (88.55327911453371, True), (74.7733952685734, True), (83.20383092008849, True), (87.46433464396905, True), (84.44733006331667, True), (79.4124817760518, True), (79.90137451753766, True), (72.66710299357113, True), (78.61265862772012, True), (87.3851459806707, True), (83.66426130596417, True), (85.49872231168618, True), (75.57831345362186, True), (81.89317960089768, True), (87.91757218636316, True), (80.1882411398938, True), (89.7845917

In [ ]:
yes = no = 0
for i in range( size_tags_single_trace ):
  if tags_trace[ 0 ][ i ] == True:
    yes += 1
  else:
    no += 1
print( "tag True", yes )
print( "tag False", no )

tag True 421848
tag False 3538154


In [ ]:
# Sliding window to create the dataset: fixing a window size and an advancement step, the aim is focusing on an Hamming weight window and tag
# with a 1 the window contains a TRUE tag, 0 otherwise
window = 500
step = 10
X_full_dataset = []
Y_full_dataset = []

for trace in range( SIZE_SIMULATION_TRACES ):
  for i in range( 0, size_weights_single_trace - window + 1, step ):
    X_full_dataset += [ weights_trace[ trace ][ i:i + window ] ]
    window_of_tags = np.array( tags_trace[ trace ][ i:i + window ] )
    set_True = 0
    # It detects where chunks begin, has some logic for the first and last chunk, and simply computes differences between chunk starts and discards lengths corresponding to False chunks
    arr = np.diff( np.where( np.concatenate( ( [ window_of_tags[ 0 ] ], window_of_tags[ :-1 ] != window_of_tags[ 1: ], [ True ] ) ) )[ 0 ] )[ ::2 ]
    if np.size( arr ) > 0 and np.max( arr ) > SO_TIME:
      set_True = 1
    Y_full_dataset += [ set_True ]

size_X_full_dataset = len( X_full_dataset )
size_Y_full_dataset = len( Y_full_dataset )
size_single_trace_X = len( X_full_dataset[ 0 ] )

print( X_full_dataset[ :10 ] )
print( Y_full_dataset[ :100 ] )
print( "Size of the X dataset of {} sliding window of Hamming weights:".format( window ), size_X_full_dataset )
print( "Size of the Y dataset of {} sliding window of tags:".format( window ), size_Y_full_dataset )
print( "Size of the first trace of the X dataset of Hamming weight in sliding window:", size_single_trace_X )

[[93.58998793425509, 82.968083592461, 76.86203580547472, 74.83062542760929, 90.75312373957557, 74.26610515980815, 83.60216673398453, 89.48087852374415, 79.80662383926291, 81.15269643859, 91.07018542029935, 77.55954342897918, 79.7554929428429, 88.54994578576365, 86.06489415451347, 77.88956937009567, 78.00712915977023, 94.58522325212785, 84.61402830386837, 69.91251429480779, 86.0137884064775, 88.55327911453371, 74.7733952685734, 83.20383092008849, 87.46433464396905, 84.44733006331667, 79.4124817760518, 79.90137451753766, 72.66710299357113, 78.61265862772012, 87.3851459806707, 83.66426130596417, 85.49872231168618, 75.57831345362186, 81.89317960089768, 87.91757218636316, 80.1882411398938, 89.78459171366481, 82.57458272644301, 84.23765875340827, 81.42904461382257, 72.18412774837836, 77.47720432933824, 93.95497279351096, 82.26004462056561, 85.18876087732754, 74.67502883846147, 90.52376560625801, 90.65348450505489, 73.36400103317351, 83.97162775147991, 87.31646284210584, 81.34002841047676, 92

In [ ]:
yes = 0
no = 0
for i in range( size_Y_full_dataset ):
  if Y_full_dataset[ i ] == 1:
    yes += 1
  else:
    no += 1
print( "finestre taggate a 1", yes )
print( "finestre taggate a 0", no )

finestre taggate a 1 49356
finestre taggate a 0 346595


In [ ]:
true = 0 # numero di true di fila
groundtruth = []

for i in range( size_Y_full_dataset ):
  if Y_full_dataset[ i ] == 1:  # Finestra corrente a 1
    true += 1
  else:                         # Finestra corrente a 0
    if true > 0:
      if true >= 15:
        if true >= 40:
          print( "True di seguito:", true, " - numero finestra: ", i, " - SUB lunga: overflow!" )
          groundtruth += [ ( 1, i ) ]   # SUB lunga
        else:
          print( "True di seguito:", true, " - numero finestra: ", i, " - SUB corta: no overflow" )
          groundtruth += [ ( 0, i ) ]
      else:
        print( "True di seguito: ", true, " - numero finestra: ", i, " - falso positivo" )
      true = 0

True di seguito: 77  - numero finestra:  77  - SUB lunga: overflow!
True di seguito: 50  - numero finestra:  272  - SUB lunga: overflow!
True di seguito: 65  - numero finestra:  481  - SUB lunga: overflow!
True di seguito: 49  - numero finestra:  1330  - SUB lunga: overflow!
True di seguito: 49  - numero finestra:  1688  - SUB lunga: overflow!
True di seguito: 187  - numero finestra:  2199  - SUB lunga: overflow!
True di seguito: 80  - numero finestra:  2751  - SUB lunga: overflow!
True di seguito: 67  - numero finestra:  3126  - SUB lunga: overflow!
True di seguito: 95  - numero finestra:  3544  - SUB lunga: overflow!
True di seguito: 52  - numero finestra:  3740  - SUB lunga: overflow!
True di seguito: 50  - numero finestra:  3934  - SUB lunga: overflow!
True di seguito: 113  - numero finestra:  4355  - SUB lunga: overflow!
True di seguito: 36  - numero finestra:  5221  - SUB corta: no overflow
True di seguito: 50  - numero finestra:  5416  - SUB lunga: overflow!
True di seguito: 66 

In [ ]:
# Load model
drive.mount( LOCAL_DRIVE, force_remount=True )

filepath = "/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/Models_trained/Model/model"
model = load_model( filepath )
print("Done.")

Mounted at /content/drive
Done.


In [ ]:
## Testing
X_full_dataset = np.array( X_full_dataset )
X_full_dataset = np.reshape( X_full_dataset, ( X_full_dataset.shape[ 0 ], X_full_dataset.shape[ 1 ], 1 ) )
Y_full_dataset = to_categorical( np.array( Y_full_dataset ) )

print( "Shapes:" )
print( "\tX_test: " + str( X_full_dataset.shape ) )
print( "\tY_test: " + str( Y_full_dataset.shape ) )

print( "Computing predictions..." )
predictions = model.predict( X_full_dataset )

print( "Evaluating model..." )
model.evaluate( X_full_dataset, Y_full_dataset )

print( "Done." )

Shapes:
	X_test: (395951, 500, 1)
	Y_test: (395951, 2)
Computing predictions...
12374/12374 [==============================] - 113s 8ms/step
Evaluating model...
12374/12374 [==============================] - 133s 11ms/step - loss: 0.1107 - accuracy: 0.9597
Done.


In [ ]:
win = 500
iter = 10
start = 0
end = 5000
for p in range( len( predictions[ start:end ] ) ):
  print( "Windows n°", p+start, " --- clocks: [", (p+start)*iter, ",", win + (p+start)*iter, "]", str( np.argmax( predictions[ p ] ) ) + " -> " + str( np.argmax( Y_full_dataset[ p ] ) ) + " [" + str( np.argmax( predictions[ p ] ) == np.argmax( Y_full_dataset[ p ] ) ) + "] \t -> " + str( np.round( predictions[ p ], 4 ) ) )

In [ ]:
fake = 0
guessed = 0
for p in range( len( predictions ) ):
  if np.argmax( predictions[ p ] ) == np.argmax( Y_full_dataset[ p ] ):
    guessed += 1
  else:
    fake += 1
print( "Quante finestre la rete ha indovinato:" + str( guessed ) )
print( "Quante finestre la rete ha sbagliato:" + str( fake ) )

Quante finestre la rete ha indovinato:380007
Quante finestre la rete ha sbagliato:15944


In [ ]:
sub_overflow_predict = []

for p in range( len( predictions ) ):
  if np.argmax( predictions[ p ] ) == 1:
    true += 1
  else:
    if true > 0:
      if true >= 15:
        if true >= 40:
          print( "True di seguito:", true, "numero predizione: ", p, "SUB lunga: overflow" )
          sub_overflow_predict += [ ( 1, p ) ]
        else:
          print( "True di seguito:", true, "numero predizione: ", p, "SUB corta: no overflow" )
          sub_overflow_predict += [ ( 0, p ) ]
      else:
        print( "True di seguito: ", true, "numero predizione: ", p )
      true = 0

In [ ]:
print( len( groundtruth ) )
groundtruth = np.array( groundtruth )
print( "Groundtruth 1:", np.count_nonzero( groundtruth == 1, axis=0 ) )
print( "Groundtruth 0:", np.count_nonzero( groundtruth == 0, axis=0 ) )
print( len( sub_overflow_predict ) )
sub_overflow_predict = np.array( sub_overflow_predict )
print( "Prediction SUB overflow 1:", np.count_nonzero( sub_overflow_predict == 1, axis=0 ) )
print( "Prediction SUB overflow 0:", np.count_nonzero( sub_overflow_predict == 0, axis=0 ) )

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation
from bokeh.models import Span
from bokeh.io import output_notebook
from bokeh.io import export_png
output_notebook()


def select_color_tf( el ):
    switch = {
        -1: 'red',
        0: 'green',
        1: 'red',
    }
    return switch.get( el, 'black' )

# Draw one example entry
trim_value = 10000
offset_value = trim_value * 0
slice_size = 10
# Figure init
p = figure( plot_width=1200, plot_height=300, title="F415 uECC scalar mult F415-NN guess T4 (slice_size: " + str( slice_size ) + ")" )
last_box_type = 2
last_box_start = 0
last_box_end = 1

# la box deve avanzare di 10 e non di 500
for index in tqdm( range( min( len( Y_full_dataset[ offset_value:offset_value+trim_value ] ), len( predictions[ offset_value:offset_value+trim_value ] ) ) ), position=0, leave=True ):
    current_operation = int( np.argmax( Y_full_dataset[ offset_value + index ] ) - np.argmax( predictions[ offset_value + index ] ) )
    #if current_operation == -1:
    #  print( "-1" )
    if index == trim_value - 1:
        # No more values
        p.add_layout( BoxAnnotation( left=( last_box_start + 1 ) * slice_size, right=( index + 1 ) * slice_size, fill_color=select_color_tf( last_box_type ) ) )
    elif not(last_box_type == current_operation):
        # Operation changed
        p.add_layout( BoxAnnotation( left=( last_box_start + 1 ) * slice_size, right=( last_box_end + 1 ) * slice_size, fill_color=select_color_tf( last_box_type ) ) )
        last_box_type = current_operation
        last_box_start = index
        last_box_end = last_box_start + 1
    else:
        # Box continues
        last_box_end += 1
p.line( range( len( traces[ 0 ][ :trim_value*slice_size ] ) ), traces[ 0 ][ :trim_value*slice_size ] )
show( p )